# Lending Club Loan Origination Data
#### Exploration, analysis, and modeling
---
**Ryan Peabody** </p>

**6 January, 2019**

---

#### Package imports

In [52]:
import numpy as np
import pandas as pd

from IPython.display import display
from pandas.api.types import CategoricalDtype

%matplotlib inline

###  Part 1: Data Exploration and Evaluation
---

The dataset of interest details Lending Club loan originations from 2007 to 2015, in a csv accessible through kaggle.com. The full dataset gives 55 features for 887,379 loans, but for our purposes we will be using the following 11 features:

> **1. `loan_amnt`** (loan amount): The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

> **2. `funded_amnt`** (funded amount): The total amount committed to that loan at that point in time.

> **3. `term`**: The number of payments on the loan. Values are in months and can be either 36 or 60.

> **4. `int_rate`** (interest rate): Interest Rate on the loan.

> **5. `grade`**: Lending Club assigned loan grade.

> **6. `annual_inc`** (annual income): The self-reported annual income provided by the borrower during registration.

> **7. `issue_d`** (issue date): The month and year in which the loan was funded.

> **8. `dti`** (debt to income ratio): A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.

> **9. `revol_bal`** (revolving balance): Total credit revolving balance.

> **10. `total_pymnt`** (total payment): Payments received to date for total amount funded.

> **11. `loan_status`** (loan status): Current status of the loan.

---
Let's begin by loading in the Lending Club loan dataset. Based on the provided definitions, we can attempt to enforce data types for some of the columns. As we read in the dataset, we'll take a quick look at the data itself, as well as try to get a sense of which columns should be categorized as numerical, categorical, or ordinal.

In [22]:
# Read in DataFrame of loan dataset, enforcing datatypes where possible
columns = {'loan_amnt': float,'funded_amnt': float, 'term': str, 'int_rate': float, 'grade': str,
           'annual_inc': float, 'issue_d': str, 'dti': float, 'revol_bal': float, 'total_pymnt': float,
           'loan_status': str}
df = pd.read_csv("loan.csv", usecols=columns.keys(), dtype=columns)

# Take an initial look at the dimensions, several rows of the loan dataset, the number of unique values in each
# column, as well as the number of NaNs in each column
df_display = pd.DataFrame(df.tail())
df_display.loc['Data type', columns.keys()] = df_display.dtypes
df_display.loc['Number of unique values', columns.keys()] = df.apply(lambda c: len(c.unique()))
df_display.loc['Number of NaNs', columns.keys()] = df.isnull().sum(axis=0)

print('')
print('Total number of rows: {}'.format(df.shape[0]))
print('Total number of features: {}'.format(df.shape[1]))
display(df_display)


Total number of rows: 887379
Total number of features: 11


,loan_amnt,funded_amnt,term,int_rate,grade,annual_inc,issue_d,loan_status,dti,revol_bal,total_pymnt
887374,10000,10000,36 months,11.99,B,31000,Jan-2015,Current,28.69,14037,3971.88
887375,24000,24000,36 months,11.99,B,79000,Jan-2015,Current,3.9,8621,9532.39
887376,13000,13000,60 months,15.99,D,35000,Jan-2015,Current,30.9,11031,3769.74
887377,12000,12000,60 months,19.99,E,64400,Jan-2015,Current,27.19,8254,3787.67
887378,20000,20000,36 months,11.99,B,100000,Jan-2015,Current,10.83,33266,7943.76
Data type,float64,float64,object,float64,object,float64,object,object,float64,float64,float64
Number of unique values,1372,1372,2,542,7,49385,103,10,4086,73740,506726
Number of NaNs,0,0,0,0,0,4,0,0,0,0,0


Now that we've seen some of the data firsthand, we can ensure that each column has the appropriate datatype, as well as investigate the four rows with missing data.

---
Numerical features read in as floats (**`loan_amnt`**, **`funded_amnt`**, **`int_rate`**, **`annual_inc`**, **`dti`**, **`revol_bal`**, **`total_pymnt`**) has been assigned the correct data type, and none are missing data

---
On the surface, **`term`** would appear to be a numerical interval variable, specifying the number of months the loan was issued for, but at regularly and evenly spaced intervals. For example, automobile loans are given over terms that only vary by twelve-month intervals. However, Lending Club only allows loans with 36 or 60-month terms (https://help.lendingclub.com/hc/en-us/articles/215466718-What-are-the-terms-of-the-loans-), allowing **`term`** to be treated as a categorical variable. Technically, **`term`** is an ordinal categorical variable, but with only two possible values, the rank of each value is mathematically unimportant.

In [57]:
df['term'] = pd.Categorical(df['term'])

**`Grade`** is an ordinal categorical variable, with ranks A-G. Lending Club actually further subdivides each grade into five subgrades (1-5), which are then used to directly calculate a lendee's final interest rate as a function of the lendee's risk of default (https://www.lendingclub.com/foliofn/rateDetail.action). We need to convert **`grade`** to a categorical variable, but preserve the rank of the loans from A-G.

In [59]:
df['grade'] = pd.Categorical(df['grade'], ordered=True)

**`Annual_inc`** contains the only missing rows in our 11-feature subset of the dataset. Let's take a look at them, as well as four other rows without missing data for comparison.

In [ ]:
# Get indices of mising rows
I = df['annual_inc'].isnull()
df_display = df.loc[I, :]
df_display = df_display.append(df.head())

# Get mean values for missing row features, as well as the whole dataset
df_display.loc['Mean of missing rows', :] = df.loc[I, :].mean()
df_display.loc['Mean of entire dataset', :] = df.mean()

display(df_display)

There is 

In [34]:
I = df.loc[:, 'loan_status'] == 'Does not meet the credit policy. Status:Fully Paid'
len(df.loc[I, :])

887379

In [4]:
df['issue_d'] = pd.to_datetime(df.loc[:5, 'issue_d']).apply(lambda dt: dt.replace(day=15))

0   2011-12-15
1   2011-12-15
2   2011-12-15
3   2011-12-15
4   2011-12-15
5   2011-12-15
Name: issue_d, dtype: datetime64[ns]

loan_amnt      0
funded_amnt    0
term           0
int_rate       0
grade          0
annual_inc     4
issue_d        0
loan_status    0
dti            0
revol_bal      0
total_pymnt    0
dtype: int64